# CSS Locators, Chaining, and Responses

Pelajari sintaksis CSS Locator dan mulailah bermain dengan ide untuk chaining (merantai) bersama CSS Locators dengan XPath. Kami juga memperkenalkan objek `Response`, yang berperilaku seperti Selectors tetapi memberi kami alat tambahan untuk memobilisasi upaya scraping di beberapa situs web.

In [73]:
# Import a scrapy Selector and request
from scrapy import Selector
import requests

## CSS Locators

### Get an "a" in this Course

In [85]:
from scrapy.http import TextResponse

url = 'https://www.datacamp.com/courses/all'
html = requests.get(url)
html = html.text

In [86]:
def how_many_elements( css ):
  sel = Selector( text = html )
  print( len(sel.css( css )) )

In [87]:
from scrapy import Selector

# Create a selector from the html (of a secret website)
sel = Selector( text = html )

# Fill in the blank
css_locator = 'div.course-block > a'

# Print the number of selected elements.
how_many_elements( css_locator )

259


### The CSS Wildcard

Anda dapat menggunakan wildcard `*` di Locators CSS juga! Bahkan, kita dapat menggunakannya dengan cara yang sama, ketika kita ingin mengabaikan jenis tag. Sebagai contoh:

* String CSS Locator `'*'` memilih semua elemen dalam dokumen HTML.
* String CSS Locator `'*.class-1'` memilih semua elemen yang termasuk `class-1`, tetapi ini tidak perlu karena string `'.class-1'` juga akan melakukan pekerjaan yang sama.
* String CSS Locator `'*#uid'` memilih elemen dengan atribut `id` yang sama dengan `uid`, tetapi ini tidak perlu karena string `'#uid'` juga akan melakukan pekerjaan yang sama.

Dalam latihan ini, kami ingin Anda bekerja dengan analogi dengan karakter wildcard yang Anda tahu dari notasi XPath untuk menemukan cara memilih semua anak dari elemen tertentu dalam notasi CSS Locator.

In [94]:
# Create the CSS Locator to all children of the element whose id is uid
css_locator = "#uid > *"

### Selectors with CSS

In [74]:
html = '''
<html>
<body>
    <div class="hello datacamp">
        <p>Hello World!</p>
    </div>
    <p>Enjoy DataCamp!</p>
</body>
</html>
'''
sel = Selector( text = html )

In [75]:
type(html)

str

In [10]:
sel.css("div > p")

[<Selector xpath='descendant-or-self::div/p' data='<p>Hello World!</p>'>]

In [11]:
sel.css("div > p").extract()

['<p>Hello World!</p>']

## Attribute and Text Selection

### You've been `href` ed

In [96]:
from scrapy import Selector

# Buat objek selector dari situs web
sel = Selector( text = html )

# Pilih semua hyperlink elemen div milik  class "course-block"
course_as = sel.css( 'div.course-block > a' )

# Memilih semua atribut href chaining dengan css
hrefs_from_css = course_as.css( '::attr(href)' )

# Memilih semua atribut href chaining dengan xpath
hrefs_from_xpath = course_as.xpath( './@href' )

In [97]:
print(hrefs_from_xpath)

[<Selector xpath='./@href' data='/courses/free-introduction-to-r'>, <Selector xpath='./@href' data='/courses/data-table-data-manipulation-r-'>, <Selector xpath='./@href' data='/courses/dplyr-data-manipulation-r-tutor'>, <Selector xpath='./@href' data='/courses/ggvis-data-visualization-r-tuto'>, <Selector xpath='./@href' data='/courses/reporting-with-r-markdown'>, <Selector xpath='./@href' data='/courses/intermediate-r'>, <Selector xpath='./@href' data='/courses/introduction-to-machine-learnin'>, <Selector xpath='./@href' data='/courses/cleaning-data-in-r'>, <Selector xpath='./@href' data='/courses/intro-to-python-for-data-scienc'>, <Selector xpath='./@href' data='/courses/intermediate-r-practice'>, <Selector xpath='./@href' data='/courses/data-visualization-with-ggplot2'>, <Selector xpath='./@href' data='/courses/data-visualization-with-ggplot2'>, <Selector xpath='./@href' data='/courses/intermediate-python-for-data-sc'>, <Selector xpath='./@href' data='/courses/data-visualization-with

### Top Level Text

In [106]:
from scrapy.http import TextResponse

url = 'https://www.DataCamp.com'
res = requests.get(url)
res = TextResponse(body=res.content, url=url)

In [107]:
def our_xpath( xpath ):
  xextr = res.xpath( xpath ).extract()
  return xextr

def our_css( css ):
  cextr = res.css( css ).extract()
  return cextr

def print_results( xpath, css_locator ):
  print( "Your XPath extracts to following:")
  print( our_xpath(xpath) )
  print("_________________\n")
  print( "Your CSS Locator extracts the following:")
  print( our_css(css_locator) )
  return None

In [108]:
# Create an XPath string to the desired text.
xpath = '//p[@id="p3"]/text()'

# Create a CSS Locator string to the desired text.
css_locator = 'p#p3::text'

# Print the text from our selections
print_results( xpath, css_locator )

Your XPath extracts to following:
[]
_________________

Your CSS Locator extracts the following:
[]


### All Level Text

In [105]:
# Create an XPath string to the desired text.
xpath = '//p[@id="p3"]//text()'

# Create a CSS Locator string to the desired text.
css_locator = 'p#p3 ::text'

# Print the text from our selections
print_results( xpath, css_locator )

Your XPath extracts to following:
[]
_________________

Your CSS Locator extracts the following:
[]


### Text Extraction

In [13]:
html = '''
<p id="p-example">
    Hello world! Try <a href="http://www.datacamp.com">DataCamp</a> today!
</p>
'''

sel = Selector( text = html )

In [14]:
sel.xpath('//p[@id="p-example"]/text()').extract()

['\n    Hello world! Try ', ' today!\n']

In [15]:
sel.xpath('//p[@id="p-example"]//text()').extract()

['\n    Hello world! Try ', 'DataCamp', ' today!\n']

In [16]:
sel.css('p#p-example::text').extract()

['\n    Hello world! Try ', ' today!\n']

In [17]:
sel.css('p#p-example ::text').extract()

['\n    Hello world! Try ', 'DataCamp', ' today!\n']

In [18]:
type(sel)

scrapy.selector.unified.Selector

## Respond

### Reveal By Response

In [109]:
from scrapy.http import TextResponse

url = 'https://www.datacamp.com/courses/all'
response = requests.get(url)
response = TextResponse(body=response.content, url=url)

In [110]:
def print_url_title( url, title ):
  print( "Here is what you found:" )
  print( "\t-URL: %s" % url )
  print( "\t-Title: %s" % title )

In [111]:
# Get the URL to the website loaded in response
this_url = response.url

# Get the title of the website loaded in response
this_title = response.xpath( '/html/head/title/text()' ).extract_first()

# Print out our findings
print_url_title( this_url, this_title )

Here is what you found:
	-URL: https://www.datacamp.com/courses/all
	-Title: Data Science Courses: R & Python Analysis Tutorials | DataCamp


### Responding with Selectors

In [112]:
# Create a CSS Locator string to the desired hyperlink elements
css_locator = 'a.course-block__link'

# Select the hyperlink elements from response and sel
response_as = response.css(css_locator)
sel_as = sel.css(css_locator)

# Examine similarity
nr = len(response_as)
ns = len(sel_as)
for i in range(min(nr, ns, 2)):
    print("Element %d from response: %s" % (i+1, response_as[i]))
    print("Element %d from sel: %s" % (i+1, sel_as[i]))
    print("")

Element 1 from response: <Selector xpath="descendant-or-self::a[@class and contains(concat(' ', normalize-space(@class), ' '), ' course-block__link ')]" data='<a class="course-block__link ds-snowplow'>
Element 1 from sel: <Selector xpath="descendant-or-self::a[@class and contains(concat(' ', normalize-space(@class), ' '), ' course-block__link ')]" data='<a class="course-block__link ds-snowplow'>

Element 2 from response: <Selector xpath="descendant-or-self::a[@class and contains(concat(' ', normalize-space(@class), ' '), ' course-block__link ')]" data='<a class="course-block__link ds-snowplow'>
Element 2 from sel: <Selector xpath="descendant-or-self::a[@class and contains(concat(' ', normalize-space(@class), ' '), ' course-block__link ')]" data='<a class="course-block__link ds-snowplow'>



### Selecting from a Selection

In [113]:
# Select all desired div elements
divs = response.css( 'div.course-block' )

# Take the first div element
first_div = divs[0]

# Extract the text from the h4 element in first_div
h4_text = first_div.css('h4::text').extract_first()

# Print out the text
print( "The text from the h4 element is:", h4_text )

The text from the h4 element is: Introduction to R


## Survey

### Titular

In [114]:
# Create a SelectorList of the course titles
crs_title_els = response.css( 'h4::text' )

# Extract the course titles 
crs_titles = crs_title_els.extract()

# Print out the course titles 
for el in crs_titles:
  print( ">>", el )

>> 
        Subscribe now. Save 50% on DataCamp and commit to learning data science and analytics.
    
>> Introduction to R
>> Data Analysis in R, the data.table Way
>> Data Manipulation in R with dplyr
>> Data Visualization in R with ggvis
>> Reporting with R Markdown
>> Intermediate R
>> Introduction to Machine Learning
>> Cleaning Data in R
>> Introduction to Python
>> Intermediate R - Practice
>> Data Visualization with ggplot2 (Part 1)
>> Data Visualization with ggplot2 (Part 2)
>> Intermediate Python for Data Science
>> Data Visualization with ggplot2 (Part 3)
>> Text Mining: Bag of Words
>> Exploring Pitch Data with R
>> Working with the RStudio IDE (Part 1)
>> Introduction to Portfolio Analysis in R
>> Writing Functions in R
>> Credit Risk Modeling in R
>> Machine Learning Toolbox
>> Working with the RStudio IDE (Part 2)
>> Joining Data in R with dplyr
>> Introduction to Databases in Python
>> Manipulating Time Series Data in R with xts & zoo
>> Introduction to Time Series Ana

In [71]:
# Calculate the number of children of the mystery element
how_many_kids = len( mystery.xpath( './*' ) )

# Print out the number
print( "The number of elements you selected was:", how_many_kids )

NameError: name 'mystery' is not defined

## Scraping For Reals

### What's the Div, Yo?

In [35]:
from scrapy import Request

response = Request(url='https://www.datacamp.com/courses/all')

In [30]:
print(type(response))
print(response.url)

<class 'scrapy.http.request.Request'>
https://www.datacamp.com/courses/all


In [37]:
from scrapy.http import TextResponse

url = 'https://www.datacamp.com/courses/all'
resp = requests.get(url)
resp = TextResponse(body=resp.content, url=url)

course_divs = resp.css('div.course-block')

In [38]:
print( len(course_divs) )

259


### Inspecting course-block

In [39]:
first_div = course_divs[0]
children = first_div.xpath('./*')
print( len(children) )

3


### The first child

In [40]:
first_div = course_divs[0]
children = first_div.xpath('./*')

In [41]:
first_child = children[0]
print( first_child.extract() )

<a class="course-block__link ds-snowplow-link-course-block" href="/courses/free-introduction-to-r">
      <div class="course-block__technology course-block__technology--r"></div>
      <div class="course-block__body">
        <h4 class="course-block__title">Introduction to R</h4>
        <p class="course-block__description">
          Master the basics of data analysis by manipulating common data structures such as vectors, matrices and data frames.
        </p>
            <div class="course-block__extra-info dc-u-fx dc-u-fx-aifs dc-u-fx-jcc dc-u-fx-fww">
              <span class="course-block__length dc-u-mh-12 dc-u-fx-center">
                
<span class="
    dc-icon 
      dc-icon--size-12
      dc-icon--primary
      dc-icon--flex dc-u-mr-8
  ">
  <svg class="dc-icon__svg">
    <use xlink:href="#clock"></use>
  </svg>
</span>

                4 hours
              </span>
            </div>
      </div>
</a>


### The second child

In [42]:
first_div = course_divs[0]
children = first_div.xpath('./*')

In [43]:
second_child = children[1]
print( second_child.extract() )

<div class="course-block__footer">
          <a class="course-block__footer-link link-unstyled" href="/courses/free-introduction-to-r">
            <div class="course-block__author">
              <img class="course-block__author-image" width="62" height="62" alt="Jonathan Cornelissen" src="https://assets.datacamp.com/users/avatars/000/464/581/square/jonathan.jpg?1469693600">
              <div class="course-block__author-body">
                <p class="course-block__author-name">Jonathan Cornelissen</p>
                <p class="course-block__author-occupation">
                  Co-founder and CEO of DataCamp
                </p>
              </div>
            </div>
</a>      </div>


### The forgotten child

In [44]:
first_div = course_divs[0]
children = first_div.xpath('./*')

In [45]:
third_child = children[2]
print( third_child.extract() )

<span class="js-mobile-progress-container js-mobile-course-progress" data-id="58" data-user-id="">
    </span>


### Listful

* In one CSS Locator

In [47]:
links = resp.css('div.course-block > a::attr(href)').extract()

* Stepwise

In [49]:
# step 1: course blocks
course_divs = resp.css('div.course-block')
# step 2: hyperlink elements
hrefs = course_divs.xpath('./a/@href')
# step 3: extract the links
links = hrefs.extract()

### Get Schooled

In [50]:
for l in links:
    print( l )

/courses/free-introduction-to-r
/courses/data-table-data-manipulation-r-tutorial
/courses/dplyr-data-manipulation-r-tutorial
/courses/ggvis-data-visualization-r-tutorial
/courses/reporting-with-r-markdown
/courses/intermediate-r
/courses/introduction-to-machine-learning-with-r
/courses/cleaning-data-in-r
/courses/intro-to-python-for-data-science
/courses/intermediate-r-practice
/courses/data-visualization-with-ggplot2-1
/courses/data-visualization-with-ggplot2-2
/courses/intermediate-python-for-data-science
/courses/data-visualization-with-ggplot2-part-3
/courses/intro-to-text-mining-bag-of-words
/courses/exploring-pitch-data-with-r
/courses/working-with-the-rstudio-ide-part-1
/courses/introduction-to-portfolio-analysis-in-r
/courses/writing-functions-in-r
/courses/introduction-to-credit-risk-modeling-in-r
/courses/machine-learning-toolbox
/courses/working-with-the-rstudio-ide-part-2
/courses/joining-data-in-r-with-dplyr
/courses/introduction-to-relational-databases-in-python
/courses/